# Time Series approach

### Methods to be used:
    - Auto Correlation Function
    - Smoothing via handcrafted Gaussian Kernel 
    - Gaussian Process Regression
    - Cross Validation 
    - Lomb Scargle (Fast and Generalized)
    - Wavelets

In [ ]:
import os
import sys
import numpy as np
import scipy
import pandas as pd
import pywt 
import matplotlib.pyplot as plt
# Fro pretty printing
import pprint
# For lag plot
from pandas.plotting import lag_plot
# For ACF
import statsmodels
from statsmodels.tsa.stattools import acf
# For zoom-in inside the plot box
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
# Gridspec
import matplotlib.gridspec as gridspec
# Vaex for lightweight plotting
import vaex as vx
# Gaussian Process
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel, WhiteKernel)
# Cross validation
from sklearn.model_selection import GridSearchCV
%matplotlib notebook

In [ ]:
try:
    import gatspy
except:
    !{sys.executable} -m pip install gatspy --user
    # !conda install --yes --prefix {sys.prefix} gatspy

### Importing data, using Kepler light curves

Import a random Kepler light curve. Time is in days, flux is a relative scale.

In [ ]:
data_list = ['DataV_koi_kplr005706966.csv', 'DataV_koi_kplr003348082.csv', 'DataV_koi_kplr003656121.csv',
        'DataV_koi_kplr004135665.csv', 'DataV_koi_kplr002302548.csv', 'DataV_koi_kplr006846911.csv',
        'DataV_koi_kplr006948054.csv', 'DataV_koi_kplr008030339.csv', 'DataV_koi_kplr008884274.csv', 
        'DataV_koi_kplr009150870.csv', 'DataV_koi_kplr009153554.csv', 'DataV_koi_kplr011442793.csv',]

In [ ]:
fname = os.path.join('lc_data', data_list[0])

In [ ]:
lc_x = pd.read_csv(fname, names=['time', 'flux', 'e_flux'], 
                   nrows=15000, engine='python')
npt_lsst = np.ceil(np.ptp(lc_x.time.values) / 1.6).astype('int')
lc_x2 = lc_x[::npt_lsst]
lc_x2.reset_index(drop=True, inplace=True)

Getting the sampling rate (cadence), we will need it after to scale measurements to day scales

In [ ]:
cad1 = np.median(np.ediff1d(lc_x['time'].values))
cad2 = np.median(np.ediff1d(lc_x2['time'].values))
print('Kepler sampling: {0:.2f} d, LSST-like sampling:{1:.2f} d'.format(cad1, cad2))

Quick visualization, both Kepler sample-like and LSST sample-like (best scenario)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(lc_x.time, lc_x.flux, marker='o', s=5, c=lc_x.e_flux, cmap='jet')
ax.scatter(lc_x2.time, lc_x2.flux, marker='.', s=50, c='lime', edgecolor='k', lw=0.5)
ax.set_xlabel('time d')
ax.set_ylabel(r'flux$_{normalized}$')
ax.set_ylim([np.min(lc_x.flux) - np.std(lc_x.flux), np.max(lc_x.flux) + np.std(lc_x.flux)])
ax.set_title('Same LC with different cadences', color='navy')

### Import light curves into vaex

In [ ]:
# Whole set
ds_all = vx.from_ascii(fname, seperator=",", names=['time', 'flux', 'e_flux'])

In [ ]:
ds_K = vx.from_pandas(lc_x, name='kplr')
ds_L = vx.from_pandas(lc_x2, name='lsst')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
# Set first subplot as active 
plt.sca(ax[0])
ds_all.scatter('time', 'flux', c='k', alpha=0.6, s=0.1, length_limit=len(ds_all))
delta_yr = np.ptp(ds_all.minmax('time')) / 365
ax[0].set_title(r'Whole LC, $\Delta$={0:.2f} yr'.format(delta_yr))
ax[0].axvline(ds_K.max('time'), color='yellowgreen', lw=2)
# Now second plot
plt.sca(ax[1])
ds_K.scatter('time', 'flux', c='dodgerblue', alpha=0.6, s=1)
ds_L.scatter('time', 'flux', c='orange', alpha=1, s=20, edgecolor='k')
delta_yr = np.ptp(ds_K.minmax('time')) / 365
ax[1].set_title(r'Subsection of $\Delta$={0:.2f} yr'.format(delta_yr), color='navy')
print(ds_K.minmax('time'))

Quick checking: lag plot should be random for structures with no memory. The correlation here is a sanity check

**This relates to... memory of the system!**

In [ ]:
plt.close('all')
fig, ax = plt.subplots(1, 2, figsize=(6, 3))
lag1 = lag_plot(lc_x.flux, ax=ax[0], marker='.', c='goldenrod', edgecolor='k', lw=0.1)
lag2 = lag_plot(lc_x2.flux, ax=ax[1], marker='.', c='dodgerblue', edgecolor='k', lw=0.1)
#
for sub in ax:
    sub.set_aspect('equal')
ax[0].set_title('Kepler sampling')
ax[1].set_title('LSST-like sampling')
plt.subplots_adjust(wspace=0.55)

## Autocorrelation Function 

It is safe to only look for periods shorter than half the light curve, $k \leq \frac{N}{2}$

Calculate the autocorrelation coefficients via statsmodels. Note that the `tsa.stattools.acf` method receives only the flux, thus assuming the spacing between each observation is uniform.

In [ ]:
acf_coeffs = acf(lc_x.flux.values, unbiased=False, nlags=len(lc_x.flux.values) // 2)
tau_k = np.arange(1, acf_coeffs.size + 1, 1)
t_d = cad1 * tau_k 
#
print('Number of coefficients from the ACF calculation is: {0}'.format(acf_coeffs.size))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
# Raw results from ACF are 'coarse', because of the nature of the input data
ax[0].scatter(tau_k, acf_coeffs, marker='.', s=10, color='navy')
# Zoom in
if 1:
    zoom_factor = 60
    ax0_zoom = zoomed_inset_axes(ax[0], zoom_factor, loc=1)
    # ax0_zoom.scatter(tau_k, acf_coeffs, marker='.', s=20, color='darkorange')
    ax0_zoom.plot(tau_k, acf_coeffs, ',-', lw=2, color='darkorange')
    ax0_zoom.set_xlim(3213, 3274)
    ax0_zoom.set_ylim(0.375, 0.383)
    ## Remove tick labels
    ax0_zoom.xaxis.set_major_formatter(plt.NullFormatter())
    ax0_zoom.yaxis.set_major_formatter(plt.NullFormatter())
    mark_inset(ax[0], ax0_zoom, loc1=2, loc2=4, fc='none', ec='goldenrod')
#
# Maxima for the coefficients
aux_maxima = scipy.signal.argrelextrema(acf_coeffs, np.greater)
ax[1].scatter(t_d, acf_coeffs, marker='.', s=10, color='lightgray')
ax[1].scatter(t_d[aux_maxima], acf_coeffs[aux_maxima], marker='^', s=20, color='lime', 
              edgecolor='k', linewidths=0.1)
#
for axis in ax:
    axis.set_ylabel(r'$\rho$', fontsize=13)
ax[0].set_xlabel(r'$\tau$', fontsize=13)
ax[1].set_xlabel(r'time $d$', fontsize=13)
#
ax[0].set_title('ACF coefficients')
ax[1].set_title('Location of local maxima')
plt.suptitle('Kepler sampling')

**Smooth the ACF coefficient distribution to easyly locate the local maxima. **

In [ ]:
def gaussian(mu, sigma, x):
    return np.exp(np.power(-(x - mu), 2) / (2 * np.power(sigma, 2))) / (sigma * np.sqrt(2 * np.pi))

Values for the Gaussian (convolution) are empirical, as a compromise between diminish noise and keep the ACF signal. 
Note we need to trim a bit the result array, due to border padding. Also, remember the normalization, to keep the scale.

A suggestion from literature (McQuillan + 2013):
```
sigma_x = 18 / 2.35
x = np.arange(0, 56, 1)
```

In [ ]:
sigma_x = 18 / 2.35
x = np.arange(0, 56, 1)
acf_g_conv = scipy.signal.convolve(acf_coeffs, gaussian(0, sigma_x, x)) / np.sum(gaussian(0, sigma_x, x))
print('Original size of the ACF coefficients array: {0}. The smoothed: {1}'.format(acf_coeffs.size, acf_g_conv.size))

In [ ]:
# Trim the padded extra section, and re-use the previously defined inteval tau_k
Ntrim = acf_g_conv.size - acf_coeffs.size
acf_g_conv = acf_g_conv[Ntrim:]

In [ ]:
# Local maxima
aux_maxima = scipy.signal.argrelextrema(acf_g_conv, np.greater)
# Global maxima
idx_gmax = np.argmax(acf_g_conv[aux_maxima])

In [ ]:
# Grid
fig = plt.figure(figsize=(5, 4))
gs1 = gridspec.GridSpec(3, 3)
gs1.update(left=0.16, right=0.98, hspace=0.05)
ax0 = fig.add_subplot(gs1[: -1, :])
ax1 = fig.add_subplot(gs1[-1, :], sharex=ax0)
#
ax1.scatter(t_d, acf_g_conv - acf_coeffs, marker='*', s=10, color='orange')
# 
ax0.scatter(t_d, acf_g_conv, marker='.', s=10, color='lightgray')
ax0.scatter(t_d[aux_maxima], acf_g_conv[aux_maxima], marker='^', s=20, color='lime', 
              edgecolor='k', linewidths=0.1)
# 
ax0.axvline(t_d[aux_maxima][idx_gmax], lw=2, c='b', alpha=0.5)
#
ax0.set_ylabel(r'$\rho$', fontsize=13)
ax1.set_xlabel(r'time $d$', fontsize=13)
ax1.set_ylabel(r'Gauss - ACF', fontsize=13)

In [ ]:
max_acf_kplr = t_d[aux_maxima][np.argmax(acf_g_conv[aux_maxima])]

In [ ]:
print('Maximum of the ACF: {0:.2f} d'.format(max_acf_kplr))

### Let's do the same for the more sparse situation...

In [ ]:
acf_coeffs_spa = acf(lc_x2.flux.values, unbiased=False, nlags=len(lc_x2.flux.values) // 2)
tau_k_spa = np.arange(1, acf_coeffs_spa.size + 1, 1)
t_d_spa = cad2 * tau_k_spa

We must remember that so far we are assuming an uniform sampling . In the case of Kepler cadence, given the amount of points, for this analysis this is not a concern.

In the case of a more sparse time series, changes in the cadence will make us result not be as accurate as if having a regular cadence.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].scatter(t_d, acf_coeffs, alpha=0.5, label='ACF for Kepler sampling', c='navy', s=10)
ax[0].scatter(t_d_spa, acf_coeffs_spa, label='ACF for downsampled data', c='orange', s=20)
# Histogram of the cadence in the data
ax[1].hist(np.ediff1d(lc_x2['time'].values), bins=10, histtype='stepfilled', color=['lemonchiffon'], lw=0)
ax[1].hist(np.ediff1d(lc_x2['time'].values), bins=10, histtype='step', color=['orange'], lw=2)
#
ax[0].legend(loc='upper right')
ax[0].set_xlabel(r'time $d$', fontsize=13)
ax[0].set_ylabel(r'$\rho$', fontsize=13)
ax[1].set_xlabel(r'$x_{(t+1)}-x_{t}$ $d$', fontsize=13)
ax[1].set_ylabel('N')
ax[0].set_title('ACF for both cadences', color='forestgreen')
ax[1].set_title('Histogram of cadence, sparse LC', color='navy')
#
plt.subplots_adjust(bottom=0.2)

In [ ]:
# Local maxima
aux_maxima_spa = scipy.signal.argrelextrema(acf_coeffs_spa, np.greater)
# Global maxima
idx_gmax = np.argmax(acf_coeffs_spa[aux_maxima_spa])

In [ ]:
max_acf_lsst = t_d_spa[aux_maxima_spa][np.argmax(acf_coeffs_spa[aux_maxima_spa])]

In [ ]:
txt = 'Maximum of the ACF, for sparse scenario: {0:.2f} d, this represents a variation'.format(max_acf_lsst)
txt +=  ' of {0:.2f}% respect to Kepler-sampling'.format(max_acf_lsst * 100 / max_acf_kplr - 100)
print(txt)

A variation of 17% maybe is within our scientific expectatins, maybe not. Let's try a method to fill gaps, for the case when we miss some observatins, or need more points.
Note that a previous knowledge of the expected behaviour increases our chances of get a more accurate prediction.

### Gap filling

Fillig the gaps will allow us to have a more regular sampled grid. In the case when fewer observations are available, compared with the main variability length, to have an uniform sampling makes calculations more stable.

**Important**: *chains of N >= 4000 and long runs, make your regression more robust. Let it run long enough!*

In [ ]:
# Instance of the GP
guess_period = max_acf_kplr
# length_scale: while larger, the shapes of the samples elongate. Default:1.
# length_scale_bounds: lower and upper bounds for lenght_scale. Let's say is a day
# periodicity_bounds: lower and upper bounds in periodicity
if 0:
    kernel = 1.0 * ExpSineSquared(length_scale=guess_period/2., 
                                  periodicity=guess_period,
                                  length_scale_bounds=(guess_period, 1.1*guess_period),
                                  periodicity_bounds=(guess_period/2, 1.5 * guess_period),
                                 )
else:
    # Exponential Sine
    expsine= 1. * ExpSineSquared(length_scale=guess_period / 2., 
                                  periodicity=guess_period,
                                  length_scale_bounds=(guess_period, 1.1*guess_period),
                                  periodicity_bounds=(guess_period/2, 1.5 * guess_period),)
    # Radial Basis Function
    rbf = 1.0 * RBF(length_scale=2 * guess_period, 
                    length_scale_bounds=(0.1 * guess_period, 4.0 * guess_period))
    # Rational Quadratic
    rquad =  1. * RationalQuadratic(length_scale=np.ptp(lc_x2.time.values) / 10, 
                                    alpha=1)
    # Matern (unstable in this scenario)
    matern = .5 * Matern(length_scale= 2 * guess_period, 
                         length_scale_bounds=(0.5 * guess_period, 10 *guess_period), 
                         nu=1.5)
    kernel = expsine + rbf
# If want to replicate the result, must use same seed 
aux_seed = np.random # 45
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50, random_state=aux_seed)
if 1:
    # Calculate the prior
    x_prior = np.linspace(lc_x2.time.values[0], lc_x2.time.values[-1], 1000)
    y_mean_prior, y_std_prior = gp.predict(x_prior[:, np.newaxis], return_std=True)
    y_samples_prior = gp.sample_y(x_prior[:, np.newaxis], 1000)

# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(lc_x2.time.values[:, np.newaxis], lc_x2.flux.values)
# Posterior
x_grid = np.linspace(lc_x2.time.values[0], lc_x2.time.values[-1], 1000)
# Make the prediction on the meshed x-axis (ask for MSE as well)
y_mean_post, y_std_post = gp.predict(x_grid[:, np.newaxis], return_std=True)
y_samples_post = gp.sample_y(x_grid[:, np.newaxis], 5000)

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4))
# Vaex for the Kepler sampling
ds_K.scatter('time', 'flux', 
             label='Kepler sampling (N={0})'.format(len(ds_K)), 
             c='navy', alpha=0.1, s=10)
# Posterior <x>
ax.plot(x_grid, y_mean_post, lw=2, c='lime', label='Mean of posterior distribution')
# Initial data points
ax.plot(lc_x2.time, lc_x2.flux, 'o', c='w', 
        label='LSST-like data points (N={0})'.format(len(lc_x2.index)), 
        markersize=4, markeredgecolor='k')
#
ax.set_title('Gaussian Process Regression result. From 15000 to 72 data points', color='navy')
ax.set_xlabel(r'time $d$', fontsize=13)
ax.set_ylabel(r'flux$_{normalized}$', fontsize=13)
plt.tight_layout()
ax.set_facecolor('floralwhite')
ax.legend(loc='upper left')

In [ ]:
t = 'Posterior\n{0}\nkernel =  {1}'.format('=' * len('Posterior'),gp.kernel_)
t += '\nLog-likelohood ={0:.3f}'.format(gp.log_marginal_likelihood(gp.kernel_.theta))
print(t)

**Note through the iterations the periodicity value was changed.**

In [ ]:
if False:
    import pickle
    pickle.dump(y_mean_post, open('y_mean_posterior_lsst.pickle', 'wb'))
    np.save('y_mean_posterior.npy', y_mean_post)

## Cross Validation

Note we trained the Gaussian Process and tested/applied it on the same dataset. In this scenario the model would probably fail to  predict unknown data. Then, we should have different subsets of data on which train and evaluate teh data.

What are this sets?
- train set: on which the training proceeds
- validation set: after the training, evaluation of trained model is made on the validation set
- test set: final evaluation of the model, when all looks good

The problem is... many times we don't have so much data to chunk into pieces. With small *N* we coud easily fall in a strong dependece of the selection of such subsets. Here is when Cross Validation comes to save us: it uses a subsample for training and then evaluate the model on the remaining data, by *folds*. 

In [ ]:
from sklearn.cross_validation import cross_val_score, KFold
# 1) Instance the GP
# Exponential Sine
expsine= 1. * ExpSineSquared(length_scale=guess_period / 2., 
                              periodicity=guess_period,
                              length_scale_bounds=(guess_period, 1.1*guess_period),
                              periodicity_bounds=(guess_period/2, 1.5 * guess_period),)
# Radial Basis Function
rbf = 1.0 * RBF(length_scale=2 * guess_period, 
                length_scale_bounds=(0.1 * guess_period, 4.0 * guess_period))
#
kernel = expsine + rbf
aux_seed = np.random 
gaussian_pr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50, random_state=aux_seed)
# 2) Fit the model, using the maximization of the likelihood
gaussian_pr.fit(lc_x2.time.values[:, np.newaxis], lc_x2.flux.values)

In [ ]:
# Define the Cross validation
gpr_grid = {
    'kernel' : [expsine + rbf],
    }
n_kfolds = 5
CV = GridSearchCV(estimator=GaussianProcessRegressor(), 
                  param_grid=gpr_grid,
                  n_jobs=2, 
                  cv=n_kfolds,
                  refit=True,
                  return_train_score=True,
                 )

In [ ]:
# Fit
CV.fit(lc_x2.time.values[:, np.newaxis], lc_x2.flux.values)

In [ ]:
# Get the prediction, note the "phantom dimension"
x_cv = np.linspace(lc_x2.time.values[0], lc_x2.time.values[-1], 1000)[:, None]
y_cv = CV.predict(x_cv)

In [ ]:
fig = plt.figure(figsize=(9, 6))
gs1 = gridspec.GridSpec(3, 3)
gs1.update(left=0.14, right=0.9, hspace=0.05)
ax0 = fig.add_subplot(gs1[: -1, :])
ax1 = fig.add_subplot(gs1[-1, :], sharex=ax0)
# Activate the first subplot
plt.sca(ax0)
# Vaex for the Kepler sampling
ds_K.scatter('time', 'flux', label='Kepler sampling (N={0})'.format(len(ds_K)), 
             c='navy', alpha=0.1, s=10)
# CV
ax0.plot(x_cv, y_cv, marker='o', color='lime', markersize=1)
#
# Initial data points
ax0.plot(lc_x2.time, lc_x2.flux, 'o', c='w', 
         label='LSST-like data points (N={0})'.format(len(lc_x2.index)), 
         markersize=4, markeredgecolor='k')

# Only neede d if vaex is going to be used: plt.sca(ax1)
# Difference
ax1.plot(x_grid, y_mean_post - y_cv, c='blueviolet', marker='*', markersize=1)
#
ax0.set_title('Gaussian Process Regression and Cross validation', color='green')
ax0.set_ylabel(r'flux$_{normalized}$', fontsize=13)
plt.tight_layout()
ax0.set_facecolor('ghostwhite')
ax0.legend(loc='upper left')
ax1.set_xlabel(r'time $d$', fontsize=13)
ax1.set_ylabel(r'posterior - cv', fontsize=13)

In [ ]:
# Results from the CV
CV_res = CV.cv_results_

In [ ]:
# Print the dictionary of the results, but in a nicer way
if 0:
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(CV_res)

## Now... a different method

## Lomb Scargle

Different modules available! **gatpsy**, **astropy**, **astroML**... and the less fancy **scipy**

#### Starting with Fast LS

In [ ]:
m_fls = gatspy.periodic.LombScargleFast(fit_period=True)

In [ ]:
m_fls.optimizer.period_range = (1, 100)

In [ ]:
m_fls.fit(lc_x.time.values, lc_x.flux.values, lc_x.e_flux.values)

First do the search on the coarse grid of frequencies, and then on a second step on a more fine grid

In [ ]:
fls_period = m_fls.best_period

To plot the periodogram

In [ ]:
periods = np.linspace(1, 100, 1000)
scores = m_fls.score(periods)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 4))
ax.plot(periods, scores)
ax.set_xlabel(r'time $d$')
ax.set_ylabel(r'periodogram power')
ax.set_title('Periodogram for the Kepler-sampled LC', fontsize=13, color='forestgreen')

Phase diagram

1) fill an auxiliary array for the predicted shape in the phase diagram. A better fit would be using the first 4 more prominent periods

2) chunk time vector to fit in the phase plot

3) plot all together

In [ ]:
aux_time = np.linspace(0, fls_period, 1000)
flux_fit = m_fls.predict(aux_time)

In [ ]:
phase = (lc_x.time.values / fls_period) % 1
phasefit = (aux_time / fls_period)

In [ ]:
ds_K_phase = vx.from_arrays(phase=phase, flux=lc_x.flux.values)

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 5))
plt.sca(ax)
kw = {
    'vmin' : 1,
    'colormap' : 'jet',
    'background_color' : 'white',
}
ds_K_phase.plot('phase', 'flux', **kw)
# ax[0].errorbar(phase, lc_x.flux.values, lc_x.e_flux.values, fmt='o')
ax.plot(phasefit, flux_fit, '-', color='w')
ax.set_xlim([0, 1])
ax.set_title('Phase plot', color='dimgray', fontweight='bold')

If we want to refine the period estimation

In [ ]:
m2_fls = gatspy.periodic.LombScargleFast(fit_period=True)
m2_fls.optimizer.set(period_range=(0.5 * fls_period, 2 * fls_period), first_pass_coverage=10)
m2_fls.fit(lc_x.time.values, lc_x.flux.values, lc_x.e_flux.values)
aux2_fls_period = m2_fls.best_period

In [ ]:
print('Optimized period: {0:.2f} d is {1} times the initial evaluation'.format(aux2_fls_period, 
                                                                               aux2_fls_period / fls_period))

For noisy light curves, a second optimization would be of special interest

#### Now let's go to the traditional LS, to get the period of the sparse case

Initialize the Lomb Scargle estimator

In [ ]:
m_genls4 = gatspy.periodic.LombScargle(Nterms=3, 
                                       fit_period=True)
m_genls1 = gatspy.periodic.LombScargle(Nterms=1, 
                                       fit_period=True)
m_genls4.optimizer.period_range = (1, 100)
m_genls1.optimizer.period_range = (1, 100)

Fit LS using the observed data

In [ ]:
m_genls4.fit(lc_x2.time.values, lc_x2.flux.values)
m_genls1.fit(lc_x2.time.values, lc_x2.flux.values)

Get best periods

In [ ]:
genls4_periods = m_genls4.find_best_periods()
genls1_periods = m_genls1.find_best_periods()

The most prominent periods:

In [ ]:
genls4_periods, genls1_periods

Periodograms, internally calculating the frequency spacing

In [ ]:
genls4_per = m_genls4.periodogram_auto()
genls1_per = m_genls1.periodogram_auto()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
ax[0].plot(genls4_per[0], genls4_per[1], ls='-', color='forestgreen')
ax[1].plot(genls1_per[0], genls1_per[1], ls='-', color='darkorange')
# Mark most prominent periods
for p in range(len(genls4_per)):
    ax[0].axvline(genls4_periods[p])
    ax[1].axvline(genls1_periods[p])
# Axis minimal setup
for subp in ax:
    subp.set_xlim([1, 50])
    subp.set_xlabel(r'period $d$', color='navy')
    subp.set_ylabel(r'power', color='navy')
ax[0].set_title('LS using 4 components for fitting')
ax[1].set_title('LS using 1 component for fitting')


### Let's try the above on new datasets!

### BONUS: Wavelet Transform

In [ ]:
time, flux = lc_x.time.values, lc_x.flux.values
s0 = 0.5
sn = 2
nscales = np.log2(sn / s0)
scales = np.arange(s0 / 2, sn, 0.01) 
m_wv = 'morl' #'morl'#'dmey' #'morl'

[cfs, frequencies] = pywt.cwt(flux, 
                              scales, 
                              m_wv, )
                              #time[1] - time[0])
power = np.power(abs(cfs), 2)
period = (1. / frequencies ) / cad1

In [ ]:
# Auxiliary vector to sum over each frequency (period)
aux_sum = np.sum(power, axis=1)
#
fig = plt.figure(figsize=(8, 8))
gs1 = gridspec.GridSpec(3, 5)
gs1.update(left=0.14, right=0.98, hspace=0.2, wspace=0.7)
ax0 = fig.add_subplot(gs1[: -1, : 3])
ax1 = fig.add_subplot(gs1[-1, : 3], sharex=ax0)
ax2 = fig.add_subplot(gs1[: -1, 3 :], sharey=ax0)
ax0.contourf(time, period, np.log2(power))
ax1.plot(time, flux)
ax2.plot(aux_sum, np.linspace(period.min(), period.max(), aux_sum.size))
#
ax0.set_yscale('log')
ax0.set_ylim([period.min(), 50])
ax0.set_ylabel(r'period $d$')
ax1.set_xlabel(r'time $d$')
ax0.set_title('Wavelet map and its LC')
ax2.set_xlabel(r'sum over period')
plt.tight_layout()